In [6]:
import pandas as pd

# Display all columns
pd.set_option('display.max_columns', None)  # or pd.set_option('display.max_columns', number_of_columns)


In [9]:
#import the audio parquet
spotify_df = pd.read_parquet("../Data/my_spotify_data/Spotify Extended Streaming History/collated_spotify_audio.parquet")

#look at first couple rows
spotify_df.head(5)

,ts,platform,ms_played,conn_country,ip_addr,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,episode_name,episode_show_name,spotify_episode_uri,audiobook_title,audiobook_uri,audiobook_chapter_uri,audiobook_chapter_title,reason_start,reason_end,shuffle,skipped,offline,offline_timestamp,incognito_mode
0,2014-02-23T00:40:19Z,Windows 7 (6.1.7601; x64; SP1; S),436960,US,134.129.174.115,Fear of the Dark,Iron Maiden,Fear of the Dark,spotify:track:0PLUzbu0ST6IOSgtUknxWV,None,None,None,None,None,None,None,unknown,trackdone,False,False,False,NaN,False
1,2014-02-23T00:42:06Z,Windows 7 (6.1.7601; x64; SP1; S),107887,US,134.129.174.115,Kings And Queens,Thirty Seconds To Mars,This Is War,spotify:track:4sjLcE0GQ6urc4iUXsUPe9,None,None,None,None,None,None,None,trackdone,unknown,False,True,False,NaN,False
2,2014-02-23T00:45:42Z,Windows 7 (6.1.7601; x64; SP1; S),215533,US,134.129.174.115,Every Night,Imagine Dragons,Night Visions,spotify:track:3WQZfz5DLbO2vnJDWNxRBf,None,None,None,None,None,None,None,unknown,trackdone,False,False,False,NaN,False
3,2014-02-23T00:46:39Z,Windows 7 (6.1.7601; x64; SP1; S),57413,US,134.129.174.115,What You Wanted,OneRepublic,Native,spotify:track:4MCuoA53lZFj4EiBFYv1B2,None,None,None,None,None,None,None,trackdone,endplay,False,True,False,NaN,False
4,2014-02-23T04:02:00Z,Windows 7 (6.1.7601; x64; SP1; S),113067,US,134.129.174.115,What You Wanted,OneRepublic,Native,spotify:track:4MCuoA53lZFj4EiBFYv1B2,None,None,None,None,None,None,None,appload,endplay,False,True,False,NaN,False
